# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

In [1]:
#Import modules
import numpy as np
import scipy.stats as stats
import pandas as pd
from statsmodels.stats.power import TTestIndPower
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
from statsmodels.formula.api import ols
import statsmodels.api as sm

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [28]:
df.id.value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [4]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [23]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

I suspect that each id that shows up twice has one click and one view, but if the id shows up once, it will only have a view and no click. I am going to verify this by looking at each of the ids that only shows up once.

In [27]:
id_norepeat = [x for x,y in zip(df.id.value_counts().index, df.id.value_counts()) if y == 1]
df_sub = df[df.id.isin(id_norepeat)]
df_sub.action.value_counts()

view    4468
Name: action, dtype: int64

As suspected, only "view" shows up in the action column for ids that only show up once.
This also verifies that there are no instances in which someone clicked without viewing.
That should be impossible anyway, but a data entry error could lead to this.

Next, I will check that the ids that show up twice have exactly one click and one view.

In [25]:
id_yesrepeat = [x for x,y in zip(df.id.value_counts().index, df.id.value_counts()) if y == 2]
no_errors = True
for i in id_yesrepeat:
    if (df[df.id == i].action.value_counts()['view'] != 1) | (df[df.id == i].action.value_counts()['click'] != 1):
        print("Unexpected counts. ID =", i)
        no_errors = False
if no_errors: print("All repeated IDs had 1 view and 1 click")

All repeated IDs had 1 view and 1 click


It appears that all the IDs that are repeated have exactly 1 view and 1 click, as expected.

Next, I will check if there is any overlap between the control and experiment groups. I only need to check the IDs that are repeated because if an ID is not repeated, it will only be in one of control or experiment.

In [57]:
no_errors = True
for i in id_yesrepeat:
    index = df[df.id == i].group.value_counts().index[0]
    if df[df.id == i].group.value_counts()[index] != 2:
        print(f"Error: ID {i} is in both control and experimental")
        no_errors = False
if no_errors: print("Each repeated ID is in just one group")

Each repeated ID is in just one group


There were no instances of one ID corresponding to both the experiment and control groups.

Below, I look at the different combinations of group and whether or not someone clicked

In [79]:
table = pd.crosstab(df.group, df.action)
table

action,click,view
group,,
control,932,3332
experiment,928,2996


Keep in mind that if someone clicked, they automatically viewed. If I want numbers for views without clicks, I need to subtract the click numbers from the corresponding view numbers.

In [80]:
table['view']['control'] = table['view']['control'] - table['click']['control']
table['view']['experiment'] = table['view']['experiment'] - table['click']['experiment']
table

action,click,view
group,,
control,932,2400
experiment,928,2068


By eye, it seems like the experiment group did click at a higher rate. Both groups include close to 930 clicks, but the experiment grouped contains only 2068 non-clicks compared to 2400 non-clicks in the control group. Let's see if the statistical test below verifies this.

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

Since there are two columns of categorical data, it is most appropriate to use a chi-squared test.

alpha = 0.05

Null Hypothesis: The proportion of people who clicked was the same for the experiment and control groups.

Alternative Hypothesis: The proportion of people who clicked was NOT the same for the experiment and control groups.

In [81]:
#Your code here
# Use built-in chi-squared function
chi2, p, dof, ex = stats.chi2_contingency(table)

print("Chi-square statistic:", chi2)
print("p-value:", p)

Chi-square statistic: 6.712921132285344
p-value: 0.00957168049704227


The p-value of the test was 0.009. This means we can reject the null hypothesis and conclude that a statistically larger proportion of people clicked on the experiment page than the control page.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [84]:
#Your code here
# Click-through rate of control group
n_control = table['click']['control'] + table['view']['control']
clickrate_control = table['click']['control'] / n_control
clickrate_control

0.2797118847539016

In [85]:
# How many would have clicked in experiment group using control click-through rate?
n_experiment = table['click']['experiment'] + table['view']['experiment']
est_clicks_experiment = clickrate_control * n_experiment
est_clicks_experiment

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [87]:
#Your code here
stdev = np.sqrt(n_experiment * clickrate_control * (1 - clickrate_control))
stdev

24.568547907005815

In [88]:
z = ( table['click']['experiment'] - est_clicks_experiment ) / stdev
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [91]:
#Your code here
p = stats.norm.sf(z)
p

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The original test (chi-squared) indicated that there is a statistically significant difference between the click-through rates of the experiment and control groups since the p-value was well below 0.05. The z-score estimate verifies this conclusion since the p-value (0.00012) is also well below 0.05. It is reasonable to conclude that the experiment homepage was more successful than the control homepage at getting people to click.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.